In [ ]:
import os
from os import environ
from typing import cast

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelInput
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings import GigaChatEmbeddings

### Examples


#### init


Нужны переменные TAGME_GIGACHAIN_TOKEN и TAGME_GIGACHAIN_BASE_URL. Клиент автоматически поднянет их. Или можно указать напрямую в kwargs клиентов или декораторов

In [3]:
token = os.environ.get("TAGME_GIGACHAIN_TOKEN", "0a3618fa-97a2-404a-b95d-15d0c3a6fa4d")
base_url = os.environ.get("TAGME_GIGACHAIN_BASE_URL", "http://127.0.0.1:8000/api")

In [ ]:
%set_env TAGME_GIGACHAIN_TOKEN=0a3618fa-97a2-404a-b95d-15d0c3a6fa4d
%set_env TAGME_GIGACHAIN_BASE_URL=http://127.0.0.1:8000/api

env: TAGME_GIGACHAIN_BASE_URL=http://127.0.0.1:8000/api
env: TAGME_GIGACHAIN_TOKEN=0a3618fa-97a2-404a-b95d-15d0c3a6fa4d


#### формируем пайплайн

In [4]:
# Создание документов
documents = [
    Document(page_content="Собаки — отличные компаньоны, известные своей преданностью."),
    Document(page_content="Кошки — независимые животные, которым нужно собственное пространство."),
    Document(page_content="Попугаи — умные птицы, способные имитировать человеческую речь."),
]

# Инициализация модели эмбеддингов
embeddings = GigaChatEmbeddings(
    credentials=environ.get("GIGA_TOKEN"),
    # base_url="https://gigachat.devices.sberbank.ru/api",
    # auth_url="https://ngw.devices.sberbank.ru:9443/api",
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP",
)

# Создание векторного хранилища
vectorstore = Chroma.from_documents(documents, embeddings)


retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # выбор наиболе подходящего результата

retriever.batch(["кошка", "акула"])

[[Document(id='8e2d040d-667a-4a62-8c44-07ab28d014b7', metadata={}, page_content='Кошки — независимые животные, которым нужно собственное пространство.')],
 [Document(id='72d8f1c2-f108-40ec-a680-6439110d67d5', metadata={}, page_content='Собаки — отличные компаньоны, известные своей преданностью.')]]

In [5]:
model = GigaChat(
    credentials=environ.get("GIGA_TOKEN"),
    verify_ssl_certs=False,
    scope="GIGACHAT_API_CORP",
)

In [6]:
message = """
Отвечай на вопросы только с помощью полученного контекста.

{question}

Контекст:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message), ("assistant", "не могу тебе помочь"), ("human", "почему?")])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model


#### Async

In [ ]:
from tagme_gigachain import tagme_trace_async


@tagme_trace_async(token=token, metadata={"model": "GigaChat-2"}, base_url=base_url)
async def run_model_async(input: LanguageModelInput) -> BaseMessage:
    return await model.ainvoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_async

response = cast(BaseMessage, await rag_chain.ainvoke("Опиши кошек двумя словами"))

response.content

'Потому что в данном контексте нет дополнительной информации, позволяющей описать кошек двумя словами. Из предоставленного описания («кошки — независимые животные, которым нужно собственное пространство») невозможно однозначно выделить именно два слова, характеризующих кошек во всей полноте их особенностей.'

#### Sync


In [10]:
from tagme_gigachain import tagme_trace


@tagme_trace(token=token, metadata={"model": "GigaChat-2"})
def run_model_sync(input: LanguageModelInput) -> BaseMessage:
    return model.invoke(input)


rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | run_model_sync

response = rag_chain.invoke("Опиши кошек одним словом")

response.content

'Потому что в представленном контексте содержится лишь информация о кошках как о независимых животных, нуждающихся в собственном пространстве. Однако одного слова недостаточно, чтобы обобщённо описать характер и поведение всех кошек. В данном случае использование единственного термина приведёт к упрощению и искажению восприятия кошек как живых существ. Если нужно конкретное слово, попробуй уточнить вопрос, указав направление описания (например: ласковые, игривые, домашние, дикие).'

#### Прямой запрос


##### Отправка диалога

In [11]:
from tagme_gigachain import TagmeIntegrationClientAsync, TagmeIntegrationClientSync

In [12]:
tagme_trace_client = TagmeIntegrationClientAsync(token)

In [19]:
dialog_delete_alarm = {
    "input": [
        {"role": "system", "content": "You are a smart home assistant."},
        {"role": "user", "content": "Удалите будильник на 7 утра."},
        {
            "role": "assistant",
            "content": None,
            "function_call": {"name": "delete_alarm_unknown", "arguments": {"time": "07:00"}},
        },
        {
            "role": "function",
            "content": None,
            "function_result": {"name": "delete_alarm_unknown", "result": {"status": "success"}},
        },
        {"role": "assistant", "content": "Будильник на 7:00 удалён."},
    ],
    "metadata": {"test": "delete_alarm_unknown"},
}

In [20]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '395800b2-b562-447d-acf2-cff46ac3696e',
 'missing': ['delete_alarm_unknown'],
 'linked_functions': [],
 'warning': True}

##### Отравка описания функции на сервер

In [28]:
function_delete_alarm = {
  "name": "delete_alarm_unknown",
  "description": "Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.",
  "parameters": {
    "properties": {
      "ids": {
        "description": "Список id будильников, которые нужно удалить",
        "items": {
          "description": "Идентификатор id будильника, который нужно удалить",
          "type": "string"
        },
        "type": "array"
      }
    },
    "required": ["ids"],
    "type": "object"
  },
  "return_parameters": {
    "description": "Ответ на delete_alarm",
    "properties": {
      "error": {
        "description": "Текст ошибки в случае, если status == fail",
        "type": "string"
      },
      "ids": {
        "description": "Список id будильников, которые удалились",
        "items": {
          "description": "Идентификатор id будильника, который удалился",
          "type": "string"
        },
        "type": "array"
      },
      "status": {
        "description": "Статус - удалось ли удалить будильник.",
        "type": "string",
        "enum": ["success", "fail"]
      }
    },
    "required": ["status"],
    "type": "object"
  },
  "few_shot_examples": []
}


In [29]:
from tagme_gigachain.types import FunctionDef


await tagme_trace_client.send_functions(
    functions=[
        FunctionDef.from_dict(function_delete_alarm)
    ]
)

In [30]:
await tagme_trace_client.get_functions()

[FunctionResponse(id=1, name='delete_alarm2', version=1, definition={'description': 'Функция удаления будильника по id. Данная функция запускается только при наличии необходимых идентификаторов id будильников в контексте. Если пользователь явно не передал id будильника, то получи метаинформацию об установленных будильниках, вызвав сначала соответствую функцию и только затем используй функцию удаления по id. Если пользователь просит удалить все будильники и в контексте диалога есть необходимые id или пользователь явно передает id будильника, который надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить будильник, сначала переспроси пользователя подтверждает ли он удаление будильника и вызывай функцию только при наличии подтверждения от пользователя.', 'parameters': {'properties': {'ids': {'description': 'Список id будильников, которые нужно удалить', 'items': {'description': 'Ид

Уже не будет missing - описание функции подвязалось с использованной в диалоге и будет отправляться разметчикам

In [31]:
await tagme_trace_client.send_dialog(dialog_delete_alarm)

{'id': '4439326a-8ff4-4814-b67d-5aae5b6e3a33',
 'missing': [],
 'linked_functions': [{'id': 4, 'name': 'delete_alarm_unknown', 'version': 1}],
 'warning': False}

#### Получение результатов и статистики разметки

In [37]:
await tagme_trace_client.get_results()

[]

In [34]:
await tagme_trace_client.get_markup_quality()

[]

In [36]:
await tagme_trace_client.get_markup_statistics()

{'task_id': '656d8511-fc9a-4d17-b608-8094b6f6ac23',
 'total_dialogs': 39,
 'marked_dialogs': 0,
 'total_markups': 0,
 'accepted_markups': 0,
 'overlap': 1}